# <span style="color:teal;">CIS 211 Project 6:  N-Body GUI</span>

##### Due 11:00 P.M. Tuesday Feb 28, 2017

##### Reading:  Online Tutorials

For this week's project you will create the elements used in a GUI for visualizing the solar system simulation: 
* a subclass of Body that adds attributes used to display a planet 
* a new type of Canvas with methods for drawing and moving planets
* a frame that contains a label and a spinbox used to control the number of planets to display
* another frame that has a run button and a place to enter the simulation parameters

###  <span style="color:teal">N-Body Simulation Code</span>

This week's project will use the Vector and Body classes from Project 4 and the `step_system` function from Project 5.  There are two ways to use that code in this project:  (1) copy your definitions from the previous projects, or (2) download the instructor's solution from Canvas.

Instructions for both methods are in the next two sections.  **Use either of these methods, but not both.**  

Note: you will earn **extra credit points** if you use your own code.

####  <span style="color:teal">Option 1: &nbsp; Use Your Own Code</span>

Choose this method only if your Vector and Body class passed all their unit tests and the `step_system` function passed its tests. Copy the complete definitions from your other notebooks and paste them into the code cells below.  

In [1]:
from math import sqrt

class Vector:
    """
    A Vector is a 3-tuple of (x,y,z) coordinates.
    """
    def __init__(self, x, y, z):
        'Creates a new vector with coordinate x, y, and z'
        self._x = x
        self._y = y
        self._z = z
        
    def __repr__(self):
        return '(%.3g,%.3g,%.3g)'%(self._x, self._y, self._z)
    
    def x(self):
        'Return the x coordinate'
        return self._x
    
    def y(self):
        'Return the y coordinate'
        return self._y
        
    def z(self):
        'Return the z coordinate'
        return self._z
    
    def norm(self):
        'Return the magnitude of the vector'
        return sqrt(self._x**2 + self._y**2 + self._z**2)
    
    def clear(self):
        'Sets all coordinates to 0'
        self._x = 0
        self._y = 0
        self._z = 0
        return Vector(self._x, self._y, self._z)
        
    def __add__(self, other):
        return Vector(self._x + other._x, self._y + other._y, self._z + other._z)
    
    def __sub__(self, other):
        return Vector(self._x - other._x, self._y - other._y, self._z - other._z)
    
    def __mul__(self, n):
        return Vector(self._x * n, self._y * n, self._z * n)
    
    def __eq__(self, other):
        return (self._x == other._x) and (self._y == other._y) and (self._z == other._z)

In [2]:
G = 6.67E-11

class Body:
    """
    A Body object represents the state of a celestial body.  A body has mass 
    (a scalar), position (a vector), and velocity (a vector).  A third vector, 
    named force, is used when calculating forces acting on a body.  An
    optional name can be attached to use in debugging.
    """
    
    def __init__(self, mass = 0, position = Vector(0,0,0), velocity = Vector(0,0,0), name = None):
        """
        Create a new Body object with the specified mass (a scalar), position (a vector), 
        and velocity (another vector).  A fourth argument is an optional name for the body.
       """
        self._name = name
        self._mass = mass
        self._pos = position
        self._vel = velocity
        self._force = Vector(0,0,0)

    def __repr__(self):
        if self._name == None:
            return '{:.3g}kg {} {}'.format(self._mass, self._pos, self._vel)
        return'{}: {:.3g}kg {} {}'.format(self._name, self._mass, self._pos, self._vel)
    
    def name(self):
        return self._name

    def mass(self):
        return self._mass

    def position(self):
        return self._pos

    def velocity(self):
        return self._vel

    def force(self):
        return self._force
    
    def direction(self, other):
        return other.position() - self.position()
    
    def add_force(self, other):
        dcubed = self.direction(other).norm() * self.direction(other).norm() * self.direction(other).norm()
        self._force = self._force + (self.direction(other) * (other.mass()/dcubed))
        return self._force
        
    def clear_force(self):
        self._force = Vector(0,0,0)
        return self._force
         
    def move(self, dt):
        acc = self.force() * G
        self._vel = self._vel + (acc * dt)
        self._pos = self._pos + (self._vel * dt)

In [3]:
def step_system(bodies, dt=86459, nsteps=1):
    orbits = [[] for i in bodies]
    for n in range(nsteps):
        x = 0
        for i in bodies:
            for j in bodies:
                if i != j:
                    i.add_force(j)
            i.move(dt)
            i.clear_force()
            orbits[x].append((i.position().x(), i.position().y()))
            x += 1
    return orbits

####  <span style="color:teal">Option 2: &nbsp; Download `Orbits.pyc` from Canvas </span>

Download one of the `Orbits.cpython-3x.pyc` files, rename it `Orbits.pyc`, and move it to the same folder as this notebook.  Then uncomment and execute the `import` command in the code cell below.

**Make sure you get the right file for your version of Python.**

In [4]:
# from Orbits import *

###  <span style="color:teal">Libraries</span>

Execute these code cells each time you open the notebook.  The cell that defines two global constants will allow us to run autograder tests when the notebook is loaded into Jupyter but not when the program is run from the command line.

In [5]:
IPython = (__doc__ is not None) and ('IPython' in __doc__)
Main    = __name__ == '__main__'

In [6]:
if IPython:
    %gui tk

In [7]:
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import filedialog
from time import sleep

###  <span style="color:teal">Helper Function</span>

To help test and debug your code execute this code cell to define a function named `read_bodies`.

The first argument passed to the function should be the name of the file containing planet descriptions.  Download `solarsystem.txt` from Canvas and save it in the same directory as this notebook.

The second argument is the type of Body object you want to make.  If you call
```
>>> read_bodies('solarsystem.txt', Body)
```
you will get back a list of Body objects (you should to this to make sure you can read the data file).  

Part 1 of this week's project is to defined a new class named TkBody that is derived from the Body class. If you call
```
>>> read_bodies('solarsystem.txt', TkBody)
```
you will get a list of TkBody objects that include the attributes needed to draw the planets on a canvas.

In [8]:
def read_bodies(filename, cls):
    '''
    Read descriptions of planets, return a list of body objects.  The first
    argument is the name of a file with one planet description per line, the
    second argument is the type of object to make for each planet.
    '''
    if not issubclass(cls, Body):
        raise TypeError('cls must be Body or a subclass of Body')

    bodies = [ ]

    with open(filename) as bodyfile:
        for line in bodyfile:
            line = line.strip()
            if len(line) == 0 or line[0] == '#':
                continue
            name, m, rx, ry, rz, vx, vy, vz, diam, color = line.split()
            args = {
                'mass' : float(m),
                'position' : Vector(float(rx), float(ry), float(rz)),
                'velocity' : Vector(float(vx), float(vy), float(vz)),
            }
            opts = {'name': name, 'color': color, 'size': int(diam)}
            for x in opts:
                if getattr(cls, x, None):
                    args[x] = opts[x]
            bodies.append(cls(**args))

    return bodies


###  <span style="color:teal">Part 1: &nbsp; TkBody (20 points)</span>

Fill in the definition of the TkBody class in the code cell below.  The constructor should accept the same arguments as the Body class constructor plus two new attributes:
* `size` is the radius, in pixels, of the circle to draw to represent the body
* `color` is a string used to defined the fill color of the circle

Define two getter methods, named `size` and `color`, to return the values of those attributes.

A third new attribute of a TkBody object is named `graphic`.  It will hold the ID of the circle created when the body is drawn. Initialize this attribute to None when the body is created, and define a "setter" named `set_graphic` and a "getter" named `graphic`.

**Note:** If you need a reminder of the arguments passed to Body type `?Body` or `help(Body)` in a Jupyter code cell.


##### <span style="color:red">Code:</span>

In [9]:
class TkBody(Body):
    
    def __init__(self, mass, position, velocity, name = '', size = 0, color = ''):
        Body.__init__(self, mass, position, velocity, name)
        self._size = size
        self._color = color
        self._graphic = None
    
    def size(self):
        return self._size
        
    def color(self):
        return self._color
        
    def graphic(self):
        return self._graphic
        
    def set_graphic(self, graphic):
        self._graphic = graphic
        

##### <span style="color:red">Tests:</span>

Use the following code cell as a "sandbox" if you want to do your own tests of the TkBody class.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [10]:
if IPython:
    b = TkBody(0, Vector(0,0,0), Vector(0,0,0), '', 0, 'black')

    assert isinstance(b, Body)
    assert isinstance(b, TkBody)

    assert b.size() == 0
    assert b.color() == 'black'
    assert b.graphic() is None

    b.set_graphic(0)
    assert b.graphic() == 0

In [11]:
if IPython:
    bodies = read_bodies('solarsystem.txt', TkBody)

    assert isinstance(bodies[0], TkBody)

    assert bodies[0].name() == 'sun'
    assert bodies[0].size() == 10
    assert bodies[0].color() == '#ffff00'

##### <span style="color:red">Documentation:</span>

In part 1, The class TkBody is a subclass from the Body class. The constructor takes in 7 arguments, passing 5 of them to the body class's constructor. TkBody's constructor then initializes the size, color, and graphic of the tkbody. there are then 3 getters for  size, color, and graphic which returns the size, color, and graphic of each. There is also 1 setter method for graphic.

###  <span style="color:teal">Part 2: &nbsp; SolarSystemCanvas (20 points)</span>

A SolarSystemCanvas is a type of Canvas used to draw planets.  An instance of this class has the following methods:
* `set_planets` is passed a list of all the bodies in the simulation; it should save this list in an instance variable for use later
* `view_planets` is passed an integer `n`, and it should draw circles representing the first `n` bodies in the list (and save the value of `n` in an instance variable)
* `move_planets` is also passed a complete list of body objects; it should use the positions of the first `n` bodies to compute new locations of the circles of the `n` planets currently on the canvas

Much of the code for this class has been written already for you.  Here are the details of the methods you need to fill in.

#### `_compute_scale` 

An important role for this class is to convert solar system coordinates (billions of meters) into screen coordinates (hundreds of pixels).  The method named `_compute_scale` calculates a scaling factor so that when a body is displayed it is placed at the correct location on the screen.  The formula is

$$f = \frac{p\,/\,2}{| \, d_\mathrm{max} \, |} $$

where $p$ is the height or width (whichever is smaller) of the canvas, in pixels, and $d_\mathrm{max}$ is the largest distance to any planet in the list passed to the method.

For example, suppose the screen is 500 pixels wide and 400 pixels tall, and this method is called with a list of 6 bodies, for the sun and all the planets up through Jupiter.  The $x$ coordinate of Jupiter at the start of the simulation is $-7.5 \times 10^{11}$, which defines a scale factor of $2.66 \times 10^{-10}$.

There are different ways to figure out the largest distance.  A simple method is to find the maximum of the norm of each position vector; another is to find the largest absolute value of any $x$ or $y$ coordinate.  

You can also include a "fudge factor" if you want to add a little extra space so the furthest orbit does not bump up against the edge of the canvas.  For example, if you use $1.1 \times d_\mathrm{max}$ in the denominator the planets will be about 10% closer to the center.

Your method should compute the scale factor and save it in an instance variable named `_scale` so it can be used later when the planets are drawn and moved.

**Important:** You need to save the scale factor in `self._scale` because that is the name used by the other methods which we have written for you.

#### `view_planets` 

This method will be passed an integer `n`, and it needs to draw circles representing the first `n` planets in the current list of TkBody objects (which have been saved in the instance variable `self._planets`).  Your method needs to:
* compute the scale factor for this list of planets
* delete any planets and lines that are on the canvas
* draw circles for the first `n` planets in `self._planets`
* save the value of `n` in `self._outer` so it can be used by `move_planets`
The circles should use the size and color attributes of the TkBody objects that represent the planets.

#### `move_planets` 

This method will be called by the main simulator after each time step.  The argument passed to the method is the list of TkBody objects with their updated position vectors.  For each planet currently on the canvas:
* get the $x$ and $y$ coordinates of the current location of the circle
* compute the $x$ and $y$ coordinates for the updated location
* move the circle to the new location
* draw a line segment from the old location to the new location

Two methods that compute screen locations have been written for you.  Call `self._current_loc(p)` to get the $x$ and $y$ coordinates of the center of the circle representing planet `p`.  Call `self._compute_loc(p)` to get the $x$ and $y$ coordinates for the new center of the circle based on the position vector of planet `p`.

In [12]:
class SolarSystemCanvas(tk.Canvas):
    
    def __init__(self, parent, height=600, width=600):
        tk.Canvas.__init__(self, parent, height=height, width=width, background='gray90', highlightthickness=0)
        self._planets = None
        self._outer = None
        self._scale = None
        self._offset = Vector(int(self['width'])/2, int(self['height'])/2, 0)
        
    def set_planets(self, lst):
        self._planets = lst
        self._outer = len(lst)
        self._compute_scale(lst)
        self.view_planets(len(lst))
        
        
    def make_circle(self,body):
        x,y = self._compute_loc(body)
        Id = self.create_oval((x - body.size(), y-body.size()),(x+body.size(), y+body.size()), fill = body.color())
        body.set_graphic(Id)

    def view_planets(self, n):
        self._scale = self._compute_scale(self._planets[:n])
        
        for j in tk.Canvas.find_all(self):
            self.delete(j)
            
        for p in self._planets[:n]:
            self.make_circle(p)
        
        self._outer = n
        
    def move_planets(self, lst):
        for p in range(self._outer):
            cur = self._planets[p]
            new = lst[p]
           
            x,y = self._current_loc(cur)
            new_x, new_y = self._compute_loc(new)
           
            self.delete(cur.graphic())
           
            self.create_line((x,y),(new_x,new_y))
            self.make_circle(new)
            
    def _compute_scale(self, bodies):
        num = float(min(self['height'], self['width'])) / 2  
        lst = [body.position().norm() for body in bodies]
        d = max(lst)
        self._scale = num / (d * 1.1)
        return self._scale
        
    def _compute_loc(self, p):
        pos = p.position() * self._scale + self._offset
        return pos.x(), pos.y()
    
    def _current_loc(self, p):
        ul, ur, _, _ = self.coords(p.graphic())
        return ul + p.size(), ur + p.size()
 

##### <span style="color:red">Tests:</span>

Use the following code cell as a "sandbox" if you want to do your own tests of the ViewFrame class.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [13]:
if IPython:
    canvas = SolarSystemCanvas(tk.Tk(), height=400, width=400)
    canvas.pack()
    bodies = read_bodies('solarsystem.txt', TkBody)

In [14]:
if IPython:
    canvas.set_planets(bodies)
    canvas.view_planets(2)
    assert [canvas.type(x) for x in canvas.find_all()].count('oval') == 2

In [15]:
if IPython:
    for i in range(10):
        step_system(bodies)
        canvas.move_planets(bodies)
    tk_objects = canvas.find_all()
    assert len(tk_objects) == 22
    assert [canvas.type(x) for x in tk_objects].count('line') == 20

##### <span style="color:red">Documentation:</span>

In part 2 the constructor initializes the canvas and other instance variables. I created a new method to make the circle of each body object that i can call to make each planet. The view_planets method simply calls the make_circle function for each body in the list and draws the line for the x and y coordinates

###  <span style="color:teal">Part 3: &nbsp; ViewControl (20 points)</span>

A ViewControl component allows the user to specify the number of planets to display on the canvas.  It's basically just a spinbox with a label, but with the extra behavior that the spinbox value can only between 2 (the canvas should always show at least the Sun and Mercury) and however many bodies are in the data set.

The constructor will be passed a reference to a callback function.  You should pass this function to the constructor that makes the Spinbox widget inside the ViewFrame.

You need to fill in the definition of the ViewFame class:
* the constructor should create a Label and a Spinbox and add them to this frame
* the `state` attribute of the Spinbox should initially be `tk.DISABLED` since we don't know yet what the maximum value is
* the `reset` method will be passed an integer `n`; it should set the spinbox lower limit to 2, set the upper limit to `n`, set the current value to `n`, and enable the spinbox by setting its state to `tk.NORMAL`
* the `nbodies` method should return 0 if the Spinbox is disable, otherwise it should return the current value in the spinbox, converted to an integer.

**Important:** &nbsp; In order to pass autograder tests for ViewControl make sure you save a reference to the Spinbox in an instance variable named `_spinbox`

##### <span style="color:red">Code:</span>

In [16]:
class ViewControl(tk.Frame):
    
    def __init__(self, parent, callback):
        tk.Frame.__init__(self, parent)
        
        self['width'] = 100             # <- the tests do not rely on these
        self['height'] = 25             # <- assignments so you can delete
        self['background'] = 'gray'     # <- them if you wish
        
        tk.Label(self, text = 'View:').grid(row=0,column=0)
        self._spinbox = tk.Spinbox(self,state = tk.DISABLED, command = callback)
        self._spinbox.grid(row=0,column=1)

    def reset(self, nbodies):
        self._spinbox.config(from_ = 2, to = nbodies, state = tk.NORMAL)
        self._spinbox.delete(0,'end')
        self._spinbox.insert(0,nbodies)
        
    def nbodies(self):
        if self._spinbox['state'] == tk.DISABLED:
            return 0
        else:
            return int(self._spinbox.get())


##### <span style="color:red">Tests:</span>

Use the following code cell as a "sandbox" if you want to do your own tests of the ViewFrame class.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [17]:
def vf_cb():
    global called
    called = True

if IPython:
    vf = ViewControl(tk.Tk(), vf_cb)
    vf.pack()        

In [18]:
if IPython:
    counts = { }
    for x in vf.children.values():
        counts.setdefault(type(x).__name__, 0)
        counts[type(x).__name__] += 1
    assert counts == { 'Label': 1, 'Spinbox': 1 }

In [19]:
if IPython:
    called = False
    vf._spinbox.invoke('buttondown')
    vf._spinbox.invoke('buttonup')
    assert not called
    
    vf.reset(5)
    assert vf._spinbox.get() == '5'
    assert vf._spinbox['from'] == 2
    assert vf._spinbox['to'] == 5

    vf._spinbox.invoke('buttondown')
    assert vf._spinbox.get() == '4'
    vf._spinbox.invoke('buttonup')
    assert vf._spinbox.get() == '5'
    
    assert called

##### <span style="color:red">Documentation:</span>

The view control class has a constructor that initializes the spinbox and creates the label  for the widget. The functions following the constructor resets the number of planets visable. nbodies shows how many planets are visable.

###  <span style="color:teal">Part 4: &nbsp; RunFrame (20 points)</span>

This project is similar to the first one, but here you will fill out the definition of a class called RunFrame.  A RunFrame is a frame that has a run button, text entry boxes for the time step size and number of steps to run, and a progress bar to show how many steps have been executed.  

The entry boxes for the time step size and number of steps should be initialized to show the default values for those parameters:  86459 for the time step size and 365 for the number of steps.

After an application has created a RunFrame object named `r` it should be able to call the following methods to interact with `r`:
* `r.dt()` returns the time step size from the time step entry box
* `r.nsteps()` returns the number of time steps indicated in that entry box
* `t.target(n)` sets the upper bound in the progress bar
* `t.update_progress(n)` sets the value to show in the progress bar

The constructor will be passed a reference to a callback function that will be called when the run button is pushed.  You should pass this function to the constructor that makes the Button.

**Note:** &nbsp; In order to test your RunFrame we need to know the names you give to some of the parts of the frame.  Make sure you use the following instance variable names:
* `_run_button` for the button that starts the simulation
* `_dt_entry` for the entry box for time step size
* `_nsteps_entry` for the entry box with the number of steps

##### <span style="color:red">Code:</span>

**Important:** &nbsp; Add your Python code to the following code cell.  Do not delete, rename, or copy this cell.

In [20]:
class RunFrame(tk.Frame):
    
    def __init__(self, parent, callback):
        tk.Frame.__init__(self, parent)
        
        self['width'] = 200
        self['height'] = 100
        self['background'] = 'gray'
        
        self._run_button = tk.Button(self, text = "Run:", command = callback)
        self._run_button.pack()
        
        self._dt_entry = tk.Entry(self)
        self._dt_entry.insert(0,'86459')
        self._dt_entry.pack()
        
        self._nsteps_entry = tk.Entry(self)
        self._nsteps_entry.insert(0,'365')
        self._nsteps_entry.pack()
        
        self._progress = ttk.Progressbar(self, orient = 'horizontal', mode = 'determinate', value = 0)
        self._progress.pack()
        
    def dt(self):
        return int(self._dt_entry.get())
    
    def nsteps(self):
        return int(self._nsteps_entry.get())
        
    def init_progress(self, n):
        self._progress['maximum'] = n
        self._progress['value'] = 0

    def update_progress(self, n):
        self._progress['value'] += n
        
    def clear_progress(self):
        self._progress['value'] = 0


In [21]:
class RunFrame(tk.Frame):
    
    def __init__(self, parent, callback):
        tk.Frame.__init__(self, parent)
        
        self['width'] = 200
        self['height'] = 100
        self['background'] = 'gray'
        
        self._run_button = tk.Button(self, text = "Run:", command = callback)
        self._run_button.grid(row=0,column=0,rowspan=2,padx=10)
        
        tk.Label(self,text='dt: ').grid(row=0,column=1)
        self._dt_entry = tk.Entry(self)
        self._dt_entry.grid(row=0,column=2,padx=10,pady=10)
        self._dt_entry.insert(0,'86459')
        
        tk.Label(self,text='nsteps: ').grid(row=1,column=1)
        self._nsteps_entry = tk.Entry(self)
        self._nsteps_entry.grid(row=1,column=2,padx=10,pady=10)
        self._nsteps_entry.insert(0,'365')
        
        self._progress = ttk.Progressbar(self, orient = 'horizontal', mode = 'determinate', value = 0)
        self._progress.grid(row=3,column=1,columnspan=2)
        
    def dt(self):
        return int(self._dt_entry.get())
    
    def nsteps(self):
        return int(self._nsteps_entry.get())
        
    def init_progress(self, n):
        self._progress['maximum'] = n
        self._progress['value'] = 0

    def update_progress(self, n):
        self._progress['value'] += n
        
    def clear_progress(self):
        self._progress['value'] = 0


##### <span style="color:red">Tests:</span>

Use the following code cell as a "sandbox" if you want to do your own tests.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [22]:
def rf_cb():
    global calls
    calls += 1

if IPython:
    rf = RunFrame(tk.Tk(), rf_cb)
    rf.pack()

In [23]:
if IPython:
    counts = { }
    for x in rf.children.values():
        counts.setdefault(type(x).__name__, 0)
        counts[type(x).__name__] += 1
    assert counts['Button'] == 1
    assert counts['Entry'] == 2
    assert counts['Progressbar'] == 1
    assert rf._dt_entry.get() == '86459'
    assert rf._nsteps_entry.get() == '365'

In [24]:
if IPython:
    calls = 0
    rf._run_button.invoke()
    rf._run_button.invoke()
    assert calls == 2

In [25]:
if IPython:
    rf._dt_entry.delete(0, tk.END)
    rf._dt_entry.insert(0, '100')
    assert rf.dt() == 100
    rf._nsteps_entry.delete(0, tk.END)
    rf._nsteps_entry.insert(0, '1000')
    assert rf.nsteps() == 1000

In [26]:
if IPython:
    rf.init_progress(100)
    for i in range(5):
        rf.update_progress(10)
    assert rf._progress['value'] == 50

##### <span style="color:red">Documentation:</span>

The constructor initializes the widgets for the time step, nstep, and progress bar. dt and nstep methods are getters that returns the integer value of dt_entry and nsteps_entry. The methods following it initialize, update, and clear the progress of the progressbar.

###  <span style="color:teal">Top Level Application</span>

The application that instantiates all the widgets and links them together is shown below. You do not need to write any code -- the program will work when all of your widget definitions are complete. If you execute this code cell in the Jupyter notebook you will get a working application that will allow you to load a set of planet definitions and run the simulatio.

Even though you do not have to write anything you should look closely at this code since it will help you understand the methods you define in each of your components.

To test your GUI, we will export the notebook to a command line application and run the application.  The graders will run the program and check the state of the canvas at the end of the simulation.

#### `load_cb` 

This function is called when the user clicks the Load button.  It uses a dialog to get the name of the file with planet definitions and reads the contents of the file to get a list of TkBody objects.  Notice how:
* the list is passed to the `set_planets` method in the SolarSystemCanvas widget, which will draw circles for all the planets
* the number of bodies is passed to the `reset` method in the ViewControl, which allows the user to click the up/down buttons to change the number of planets displayed

#### `view_cb` 

This function is the callback for the ViewControl component.  Whenever the user changes the value of the counter the new value is passed to `view_planets` so the canvas is updated to show the new number of planets.

#### `run_cb` 

This function is the one that runs the simulation.  It is invoked when the user clicks the Run button in the RunFrame widget.  It gets the simulation parameters from the widget and then calls the `time_step` function to start the simulation.

There body of the `time_step` function could just have a for loop that calls `step_system` for the specified number of time steps.  But since `run_cb` is a callback, Tk won't update the display until the callback is done, which means we won't see the planets move until after the last time step.

The technique implemented here is pretty common in GUI programs.  It runs one time step and then uses the `after_idle` function to schedule a call to run the next time step 0.02 seconds later.  Once the next call is scheduled the callback exits and returns control to the GUI and we will see the planets moving.

In [27]:
root = tk.Tk()
root.title("Solar System")

bodies = None

def load_cb():
    global bodies
    fn = tk.filedialog.askopenfilename()
    bodies = read_bodies(fn, TkBody)
    canvas.set_planets(bodies)
    view_count.reset(len(bodies))

def view_cb():
    canvas.view_planets(view_count.nbodies())
    
def run_cb():
    
    def time_step():
        nonlocal nsteps
        step_system(bodies, dt)
        canvas.move_planets(bodies)
        run_frame.update_progress(1)
        sleep(0.02)
        if nsteps > 0:
            nsteps -= 1
            canvas.after_idle(time_step)
        else:
            run_frame.clear_progress()
        
    nsteps = run_frame.nsteps()
    run_frame.init_progress(nsteps)
    dt = run_frame.dt()
    canvas.after_idle(time_step)

canvas = SolarSystemCanvas(root)
canvas.grid(row = 0, column = 0, columnspan = 3, padx = 10, pady = 10, sticky="nsew")

tk.Button(root, text='Load', command=load_cb).grid(row=1, column=0, pady = 20)

view_count = ViewControl(root, view_cb)
view_count.grid(row=1, column=1, pady=20)

run_frame = RunFrame(root, run_cb)
run_frame.grid(row=1, column=2, pady=20)

if Main and not IPython:
    try:
        bodies = read_bodies("solarsystem.txt", TkBody)
        canvas.set_planets(bodies)
        view_count.reset(len(bodies))
        for i in range(5):
            view_count._spinbox.invoke('buttondown')
        run_frame._nsteps_entry.delete(0, tk.END)
        run_frame._nsteps_entry.insert(0,'100')
        root.update()
        run_frame._run_button.invoke()
    except Exception as err:
        print(err)
    input('hit return to continue...')


##### <span style="color:red">Simulation Score</span>

**Important:** &nbsp; You do not have to write anything for this part of the project.  Leave the cell below empty, in spite of the fact that is says "your documentation here".

The graders will use the cell to write comments about your GUI and to enter the grade for your solar system simulation.  **Do not edit, move, rename, or delete the following cell.**

YOUR DOCUMENTATION HERE